In [ ]:
import pandas as pd
df = pd.read_csv("/content/chatgpt_reviews.csv")


In [ ]:
df.head()

,date,title,review,rating
0,2023-05-21 16:42:24,Much more accessible for blind users than the ...,Up to this point I’ve mostly been using ChatGP...,4
1,2023-07-11 12:24:19,"Much anticipated, wasn’t let down.",I’ve been a user since it’s initial roll out a...,4
2,2023-05-19 10:16:22,"Almost 5 stars, but… no search function",This app would almost be perfect if it wasn’t ...,4
3,2023-05-27 21:57:27,"4.5 stars, here’s why","I recently downloaded the app and overall, it'...",4
4,2023-06-09 07:49:36,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4


In [ ]:
df.isnull().sum()


,0
date,0
title,0
review,0
rating,0


In [ ]:
df = pd.get_dummies(df, columns = ["title"], dtype="int64", drop_first= True)

In [ ]:
df.head()

,date,review,rating,"title_""Powerful Software with a Simple Interface, but Room for Improvement on iPad 12 Pro""","title_""Version error?""",title_(GPT Plus user) Request failed. Please try again later.,title_(error=enforcement_failed),title_-,title_.,title_1,...,title_👌,title_👍,title_👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍,title_💸,title_🙌🏻,title_🙏,title_🤔,title_🤖 💡🌐,title_🤠👍,title_🥹
0,2023-05-21 16:42:24,Up to this point I’ve mostly been using ChatGP...,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-07-11 12:24:19,I’ve been a user since it’s initial roll out a...,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-05-19 10:16:22,This app would almost be perfect if it wasn’t ...,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-05-27 21:57:27,"I recently downloaded the app and overall, it'...",4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-06-09 07:49:36,I appreciate the devs implementing Siri suppor...,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import gradio as gr

In [ ]:


# --- 1. Model Training ---

# Ensure NLTK data is available
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('punkt')

# Create a sample DataFrame
data = {
    'review': [
        "ChatGPT is an amazing and revolutionary AI tool! So helpful for my work.",
        "The servers are always down. It's so frustrating and unreliable.",
        "It's a decent tool, but sometimes the answers are too generic.",
        "I love using it for brainstorming ideas. Absolutely fantastic!",
        "Completely useless. The information it gave was factually incorrect.",
        "It's okay for simple questions but struggles with complex topics.",
        "The best productivity booster I have ever used. A real game changer.",
        "Very slow responses and frequent network errors. Bad experience."
    ],
    'rating': [5, 1, 3, 5, 1, 3, 5, 2]
}
df = pd.DataFrame(data)

# Create Sentiment Labels
def assign_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'
df['sentiment'] = df['rating'].apply(assign_sentiment)

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_review'] = df['review'].apply(preprocess_text)

# Vectorize and Train the model
X = df['processed_review']
y = df['sentiment']
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(X)
model = LogisticRegression()
model.fit(X_tfidf, y)

print("Model training complete! ✅")

# --- 2. The Prediction Function ---

def predict_sentiment(text):
    processed_text = preprocess_text(text)
    vectorized_text = tfidf_vectorizer.transform([processed_text])
    prediction_proba = model.predict_proba(vectorized_text)
    sentiment_probabilities = dict(zip(model.classes_, prediction_proba[0]))
    return sentiment_probabilities

# --- 3. Create and Launch the Gradio Interface ---

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=5, label="Enter Your Review Here", placeholder="e.g., 'This AI is incredibly helpful!'"),
    outputs=gr.Label(label="Sentiment Score"),
    title="🤖 ChatGPT Review Sentiment Analyzer",
    description="Type in a review for ChatGPT and see the model's sentiment prediction.",
    examples=[
        ["The responses are fast and accurate."],
        ["It frequently crashes and gives error messages."],
        ["It's useful for some things but not for everything."]
    ]
)

# Launch the app with a public link
iface.launch(share=True) # The key change for Colab!import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import gradio as gr

# --- 1. Model Training ---

# Ensure NLTK data is available
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('punkt')

# Create a sample DataFrame
data = {
    'review': [
        "ChatGPT is an amazing and revolutionary AI tool! So helpful for my work.",
        "The servers are always down. It's so frustrating and unreliable.",
        "It's a decent tool, but sometimes the answers are too generic.",
        "I love using it for brainstorming ideas. Absolutely fantastic!",
        "Completely useless. The information it gave was factually incorrect.",
        "It's okay for simple questions but struggles with complex topics.",
        "The best productivity booster I have ever used. A real game changer.",
        "Very slow responses and frequent network errors. Bad experience."
    ],
    'rating': [5, 1, 3, 5, 1, 3, 5, 2]
}
df = pd.DataFrame(data)

# Create Sentiment Labels
def assign_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'
df['sentiment'] = df['rating'].apply(assign_sentiment)

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_review'] = df['review'].apply(preprocess_text)

# Vectorize and Train the model
X = df['processed_review']
y = df['sentiment']
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(X)
model = LogisticRegression()
model.fit(X_tfidf, y)

print("Model training complete! ✅")

# --- 2. The Prediction Function ---

def predict_sentiment(text):
    processed_text = preprocess_text(text)
    vectorized_text = tfidf_vectorizer.transform([processed_text])
    prediction_proba = model.predict_proba(vectorized_text)
    sentiment_probabilities = dict(zip(model.classes_, prediction_proba[0]))
    return sentiment_probabilities

# --- 3. Create and Launch the Gradio Interface ---

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=5, label="Enter Your Review Here", placeholder="e.g., 'This AI is incredibly helpful!'"),
    outputs=gr.Label(label="Sentiment Score"),
    title="🤖 ChatGPT Review Sentiment Analyzer",
    description="Type in a review for ChatGPT and see the model's sentiment prediction.",
    examples=[
        ["The responses are fast and accurate."],
        ["It frequently crashes and gives error messages."],
        ["It's useful for some things but not for everything."]
    ]
)

# Launch the app with a public link
iface.launch(share=True) # The key change for Colab!

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Model training complete! ✅
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://711be41ed080220000.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Model training complete! ✅
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fac774af13aa440f7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
